###  Created by Luis A. Sanchez-Perez (alejand@umich.edu).
<p><span style="color:green"><b>Copyright &#169;</b> Do not distribute or use without authorization from author.</span></p>

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from scipy.stats import norm 
from scipy.special import logsumexp

In [2]:
# Load dataset
dataset = datasets.load_iris()
print(dataset.feature_names, end="\n")
print(dataset.target_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


In [3]:
# Splitting
X_train, X_test, y_train, y_test = train_test_split(dataset.data,dataset.target)

### Using sklearn implementation

In [4]:
# Fitting Naive Bayes
classifier = GaussianNB()
classifier.fit(X_train,y_train)
print(classifier.theta_)
print(classifier.sigma_)

[[5.03055556 3.44722222 1.46944444 0.25833333]
 [5.96052632 2.77894737 4.28947368 1.33684211]
 [6.58421053 2.98684211 5.53157895 2.04210526]]
[[0.13767747 0.15693673 0.03323303 0.011875  ]
 [0.28291552 0.09218837 0.23146815 0.04390582]
 [0.4297507  0.09272161 0.31742383 0.05612189]]


In [5]:
# Predicting the training set results
y_pred = classifier.predict(X_train)
cm = confusion_matrix(y_train, y_pred)
print(cm)
print(accuracy_score(y_train,y_pred))

[[36  0  0]
 [ 0 36  2]
 [ 0  2 36]]
0.9642857142857143


In [6]:
# Predicting the test set results
y_pred = classifier.predict(X_test,)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test,y_pred))

[[14  0  0]
 [ 0 12  0]
 [ 0  2 10]]
0.9473684210526315


### Creating new input

In [7]:
point = np.array([2,1,3,1],ndmin=2)

### Custom implementation

In [8]:
# Computing sklearn model output to compare
classifier.predict_proba(point)

array([[1.58454718e-26, 9.99999911e-01, 8.93546989e-08]])

#### Building mean and std matrices
Notice here we are using list comprehension. Notice we work on the training set only. We build a mean and a std matrix where each row represent the values to define the likelihood pdf for each class.

In [9]:
mean = np.array([X_train[y_train == i,:].mean(axis=0) for i in np.unique(y_train)])
mean

array([[5.03055556, 3.44722222, 1.46944444, 0.25833333],
       [5.96052632, 2.77894737, 4.28947368, 1.33684211],
       [6.58421053, 2.98684211, 5.53157895, 2.04210526]])

In [10]:
std = np.array([X_train[y_train == i,:].std(axis=0) for i in np.unique(y_train)])
std

array([[0.37104915, 0.39615241, 0.18229927, 0.10897247],
       [0.53189803, 0.30362537, 0.48111136, 0.20953715],
       [0.65555373, 0.30450223, 0.56340378, 0.23690058]])

#### Building priors vector
We use frequency analysis to determine this. Notice we work on the training set only. We get one value per class.

In [11]:
prior = [(y_train == i).sum()/len(y_train) for i in np.unique(y_train)]
prior

[0.32142857142857145, 0.3392857142857143, 0.3392857142857143]

#### Computing the likelihood
We compute the likelihood of observing each feature value in the given input (this uses the proper pdf, that is the proper mean and std from the matrix). The result is a matrix the same size as the mean and std matrices.

In [12]:
likelihood = norm.pdf(point,mean,std)
likelihood

array([[3.51476977e-15, 5.20528731e-09, 1.07990954e-15, 3.19872933e-10],
       [6.85050801e-13, 4.61651634e-08, 2.28452396e-02, 5.22987420e-01],
       [1.46457667e-11, 7.45530057e-10, 2.92291168e-05, 1.05790915e-04]])

#### Computing the log-likelihood instead
We compute the log likelihood. As you can see the values are not as small as the ones from the likelihood so we have less chance to underflow.

In [13]:
loglikelihood = np.log(likelihood)
loglikelihood

array([[-33.28180237, -19.07359094, -34.46189912, -21.86309728],
       [-28.0092834 , -16.89104036,  -3.77901252,  -0.64819787],
       [-24.94686979, -21.01692566, -10.4403452 ,  -9.15404592]])

#### Computing the posterior
Here we first use the straight foward method and along the way we can observe how we get really small values and of course a bigger chance to underflow.

In [14]:
numerator = likelihood.prod(axis=1) * prior
numerator

array([2.03137890e-48, 1.28200491e-22, 1.14553116e-29])

In [15]:
lognumerator = loglikelihood.sum(axis=1) + np.log(prior)
lognumerator

array([-109.81536964,  -50.40844686,  -66.63909928])

In [16]:
posterior = numerator/numerator.sum()
posterior

array([1.58453271e-26, 9.99999911e-01, 8.93546540e-08])

In [17]:
posterior.sum()

1.0

#### Computing the log-posterior
We compute the log posterior using two different ways to determine P(x). First we compute this in an 'unsafe' (less stable) way and then we use te logsumexp trick.

In [18]:
numerator.sum()

1.2820050265870852e-22

In [19]:
logposterior = lognumerator - np.log(numerator.sum()) # unsafe
logposterior

array([-5.94069229e+01, -8.93546570e-08, -1.62306525e+01])

In [20]:
posterior = np.exp(logposterior)
posterior

array([1.58453271e-26, 9.99999911e-01, 8.93546540e-08])

In [21]:
posterior.sum()

1.0000000000000009

In [22]:
logposterior = lognumerator - logsumexp(lognumerator) # more stable
logposterior 

array([-5.94069229e+01, -8.93546570e-08, -1.62306525e+01])

In [23]:
posterior = np.exp(logposterior)
posterior

array([1.58453271e-26, 9.99999911e-01, 8.93546540e-08])

In [24]:
posterior.sum()

1.0000000000000009